In [2]:
import tensorflow as tf
import cv2 as cv

print('tf: '+tf.__version__)
print('cv: '+cv.__version__)

tf: 2.6.1
cv: 4.5.4


In [4]:
cap = cv.VideoCapture('../hmdb51_org/fall_floor/50_FIRST_DATES_fall_floor_f_cm_np1_fr_med_14.avi')
while cap.isOpened():
    ret, frame=cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting...")
        break
    cv.imshow('frame',frame)
    if cv.waitKey(100)==ord('q'):
        break

cap.release()
cv.destroyAllWindows()

Can't receive frame (stream end?). Exiting...


In [13]:
import cv2 as cv
actions = os.listdir('../hmdb51_org')

for video_name in os.listdir('../hmdb51_org/fall_floor'):
   # try:
        cap=cv.VideoCapture('../hmdb51_org/fall_floor/'+video_name)
        while cap.isOpened():
            ret, frame=cap.read()
            if not ret:
                print("Can't receive frame (stream end?). Exiting...")
                break
            cv.imshow('frame',frame)
            if cv.waitKey(1)==ord('q'):
                break

        cap.release()
        cv.destroyAllWindows()
  #  except Exception as e:
   #     print(f'Failed on {video_name}')

Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?). Exiting...
Can't receive frame (stream end?).

KeyboardInterrupt: 